# **Waste Material Segregation for Improving Waste Management**

## **Objective**

The objective of this project is to implement an effective waste material segregation system using convolutional neural networks (CNNs) that categorises waste into distinct groups. This process enhances recycling efficiency, minimises environmental pollution, and promotes sustainable waste management practices.

The key goals are:

* Accurately classify waste materials into categories like cardboard, glass, paper, and plastic.
* Improve waste segregation efficiency to support recycling and reduce landfill waste.
* Understand the properties of different waste materials to optimise sorting methods for sustainability.

## **Data Understanding**

The Dataset consists of images of some common waste materials.

1. Food Waste
2. Metal
3. Paper
4. Plastic
5. Other
6. Cardboard
7. Glass


**Data Description**

* The dataset consists of multiple folders, each representing a specific class, such as `Cardboard`, `Food_Waste`, and `Metal`.
* Within each folder, there are images of objects that belong to that category.
* However, these items are not further subcategorised. <br> For instance, the `Food_Waste` folder may contain images of items like coffee grounds, teabags, and fruit peels, without explicitly stating that they are actually coffee grounds or teabags.

## **1. Load the data**

Load and unzip the dataset zip file.

**Import Necessary Libraries**

In [12]:
# Recommended versions:

# numpy version: 1.26.4
# pandas version: 2.2.2
# seaborn version: 0.13.2
# matplotlib version: 3.10.0
# PIL version: 11.1.0
# tensorflow version: 2.18.0
# keras version: 3.8.0
# sklearn version: 1.6.1

In [2]:
# Import essential libraries
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import gdown
import zipfile
import os
# Suppressing Warnings
import warnings

ImportError: Traceback (most recent call last):
  File "C:\Users\premc\AppData\Roaming\Python\Python312\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [ ]:
print("numpy version:", np.__version__)
print("pandas version:",pd.__version__)
print("seaborn version",sb.__version__)
print("matplotlib version:",mlib.__version__)
print("PIL version",PIL.__version__)
print("tensorflow",tf.__version__)
print("keras",ks.__version__)
print("sklearn",sk.__version__)

Load the dataset.

In [ ]:
# Load and unzip the dataset
# Path to the zip file

zip_path = r'Downloads/data.zip'

# Open the zip file
extract_dir = r'E:\CNN_Project\Course 4 DL\data'  

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)
        

## **2. Data Preparation** <font color=red> [25 marks] </font><br>


### **2.1 Load and Preprocess Images** <font color=red> [8 marks] </font><br>

Let us create a function to load the images first. We can then directly use this function while loading images of the different categories to load and crop them in a single step.

#### **2.1.1** <font color=red> [3 marks] </font><br>
Create a function to load the images.

In [ ]:
# Create a function to load the raw images

def load_images_from_folders(base_dir):
    images = []
    labels = []
    for label in os.listdir(base_dir):
        class_dir = os.path.join(base_dir, label)
        if os.path.isdir(class_dir):
            for img_file in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_file)
                try:
                    img = Image.open(img_path).convert('RGB')
                    images.append(img)
                    labels.append(label)
                except:
                    continue
    return images, labels

#### **2.1.2** <font color=red> [5 marks] </font><br>
Load images and labels.

Load the images from the dataset directory. Labels of images are present in the subdirectories.

Verify if the images and labels are loaded correctly.

In [ ]:
# Get the images and their labels

base_folder = os.path.join(extract_dir, 'data')  # Adjust if necessary
images, labels = load_images_from_folders(base_folder)



Perform any operations, if needed, on the images and labels to get them into the desired format.

### **2.2 Data Visualisation** <font color=red> [9 marks] </font><br>

#### **2.2.1** <font color=red> [3 marks] </font><br>
Create a bar plot to display the class distribution

In [ ]:
# Visualise Data Distribution
label_df = pd.DataFrame(labels, columns=['Label'])
plt.figure(figsize=(10,5))
sns.countplot(data=label_df, x='Label')
plt.title("Class Distribution")
plt.xticks(rotation=45)
plt.show()


Data has significant imbalance (e.g., Plastic has 4x more samples than cardboard, 3x more samples than Glass, approx 2.5x more samples than metal, paper etc.)

#### **2.2.2** <font color=red> [3 marks] </font><br>
Visualise some sample images

In [ ]:
# Visualise Sample Images (across different labels)
def show_sample_images(images, labels, n=7):
    plt.figure(figsize=(15, 5))
    classes = list(set(labels))
    shown = set()
    count = 0
    for i, (img, lbl) in enumerate(zip(images, labels)):
        if lbl not in shown:
            plt.subplot(1, n, count+1)
            plt.imshow(img)
            plt.title(lbl)
            plt.axis('off')
            shown.add(lbl)
            count += 1
        if count == n:
            break
    plt.show()

show_sample_images(images, labels)


#### **2.2.3** <font color=red> [3 marks] </font><br>
Based on the smallest and largest image dimensions, resize the images.

In [ ]:
# Find the smallest and largest image dimensions from the data set
widths, heights = zip(*(img.size for img in images))
print(f"Smallest image size: {min(widths)}x{min(heights)}")
print(f"Largest image size: {max(widths)}x{max(heights)}")


In [ ]:
# Resize the image dimensions
# Resize and preprocess
img_size = (128, 128)
resized_images = [img.resize(img_size) for img in images]
image_array = np.array([np.array(img) for img in resized_images]) / 255.0


### **2.3 Encoding the classes** <font color=red> [3 marks] </font><br>

There are seven classes present in the data.

We have extracted the images and their labels, and visualised their distribution. Now, we need to perform encoding on the labels. Encode the labels suitably.

####**2.3.1** <font color=red> [3 marks] </font><br>
Encode the target class labels.

In [ ]:
# Encode the labels suitably

# Encode the labels
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)
labels_cat = to_categorical(labels_encoded)

### **2.4 Data Splitting** <font color=red> [5 marks] </font><br>

#### **2.4.1** <font color=red> [5 marks] </font><br>
Split the dataset into training and validation sets

In [ ]:
# Assign specified parts of the dataset to train and validation sets

X_train, X_val, y_train, y_val = train_test_split(image_array, labels_cat, test_size=0.2, stratify=labels_encoded, random_state=42)

## **3. Model Building and Evaluation** <font color=red> [20 marks] </font><br>

### **3.1 Model building and training** <font color=red> [15 marks] </font><br>

#### **3.1.1** <font color=red> [10 marks] </font><br>
Build and compile the model. Use 3 convolutional layers. Add suitable normalisation, dropout, and fully connected layers to the model.

Test out different configurations and report the results in conclusions.

In [ ]:
# Build and compile the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


#### **3.1.2** <font color=red> [5 marks] </font><br>
Train the model.

In [ ]:
import sys
print(f"Image array size: {sys.getsizeof(image_array)/1e6:.2f} MB")
print(f"Labels size: {sys.getsizeof(labels_cat)/1e6:.2f} MB")

Use appropriate metrics and callbacks as needed.

In [ ]:
# import dask.array as da

# image_array_dask = da.from_array(image_array, chunks=(1000, -1, -1, -1))  # adjust chunk size
# labels_cat_dask = da.from_array(labels_cat, chunks=(1000, -1))

# # Use Dask's train_test_split
# X_train, X_val, y_train, y_val = train_test_split(image_array_dask, labels_cat_dask, test_size=0.2, stratify=labels_encoded, random_state=42)

from sklearn.model_selection import train_test_split
import numpy as np

indices = np.arange(len(image_array))
train_idx, val_idx = train_test_split(indices, test_size=0.2, stratify=labels_encoded, random_state=42)

# Instead of copying data, access with indices
X_train = image_array[train_idx]
y_train = labels_cat[train_idx]
X_val = image_array[val_idx]
y_val = labels_cat[val_idx]


In [ ]:
# Training
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), batch_size=16)

### **3.2 Model Testing and Evaluation** <font color=red> [5 marks] </font><br>

#### **3.2.1** <font color=red> [5 marks] </font><br>
Evaluate the model on test dataset. Derive appropriate metrics.

In [ ]:
# Evaluate on the test set; display suitable metrics

# Evaluate
y_pred_probs = model.predict(X_val)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_val, axis=1)

print("Classification Report:\n")
print(classification_report(y_true, y_pred, target_names=le.classes_))

## **4. Data Augmentation** <font color=red> [optional] </font><br>

#### **4.1 Create a Data Augmentation Pipeline**

##### **4.1.1**
Define augmentation steps for the datasets.

In [ ]:
# Define augmentation steps to augment images

# Augmentation
augmentor = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)


Augment and resample the images.
In case of class imbalance, you can also perform adequate undersampling on the majority class and augment those images to ensure consistency in the input datasets for both classes.

Augment the images.

In [ ]:
# Create a function to augment the images
# Create a function to augment the images
def create_augmented_generator(X, y, batch_size=32):
    return augmentor.flow(X, y, batch_size=batch_size)



In [ ]:
# Create the augmented training dataset
augmented_train_gen = create_augmented_generator(X_train, y_train)


##### **4.1.2**

Train the model on the new augmented dataset.

In [ ]:
# Train the model using augmented images

history_aug = model.fit(augmented_train_gen, epochs=10, validation_data=(X_val, y_val), steps_per_epoch=len(X_train)//32)

## **5. Conclusions** <font color = red> [5 marks]</font>

#### **5.1 Conclude with outcomes and insights gained** <font color =red> [5 marks] </font>

* Report your findings about the data
* Report model training results

Report your findings about the data

Data has significant imbalance (e.g., Plastic has 3x more samples than Glass, approx 2.5x more samples than metal, paper etc.), this can lead to biased training.
Report model training results

#### **Analysis on Training done on the new augmented dataset**####
Improved Generalization Validation accuracy has increased from ~34% → 43%, showing that augmentation helped reduce overfitting and improved robustness.

Consistent Drop in Validation Loss Validation loss decreased steadily from 1.68 to 1.47, indicating the model is learning meaningful patterns from augmented data.

Narrower Gap Between Training & Validation Accuracy

(i) No significant overfitting: training and validation accuracies are fairly close (good balance).

(ii) Indicates that augmentation is preventing memorization and encouraging general learning.

##### **Areas for Further Improvement** ####
1. Inconsistent Training Accuracy

(i) Training accuracy fluctuates (e.g., drops in Epoch 2 & 4), possibly due to

     a. Heavy augmentation

     b. Small batch size or learning rate instability
(ii) We can sonsider using learning rate schedulers or less aggressive augmentations.

Plateauing Accuracy

(i) Val accuracy flattens around 43–44% toward the end — model might be reaching its capacity.

(ii) We can improve architecture or try transfer learning.

Class-Wise Impact Not Visible Yet

While overall accuracy improved, it’s still important to check the confusion matrix and classification report to verify if all classes are now being predicted better — not just “Food_Waste”.